## TradingEnv-v0

### Open AI 'Gym' for reinforcement-learning based trading algorithms

This gym implements a very simple trading environment for reinforcement learning.

The gym provides daily observations based on real market data pulled from Quandl on, by default, the SPY etf.  An episode is defined as 252 contiguous days sampled from the overall dataset.  Each day is one 'step' within the gym and for each step, the algo has a choice:

 - SHORT (0)
 - FLAT (1)
 - LONG  (2)
 
If you trade, you will be charged, by default, 10 BPS of the size of your trade.  Thus, going from short to long costs twice as much as going from short to/from flat.  Not trading also has a default cost of 1 BPS per step.  Nobody said it would be easy!
 
At the beginning of your episode, you are allocated 1 unit of cash.  This is your starting Net Asset Value (NAV). 

### Beating the trading game 

For our purposes, we'll say that beating a buy & hold strategy, on average, over one hundred episodes will notch a win to the proud ai player.  We'll illustrate exactly what that means below.

### Let's look at some code using the environment


###  imports

In [ ]:
import gym
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
import gym_trading

env = gym.make('trading-v0')
#env.time_cost_bps = 0 # 

env = env.unwrapped



import tensorflow as tf
import policy_gradient

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: Environment '<class 'gym_trading.envs.trading_env.TradingEnv'>' has deprecated methods. Compatibility code invoked.


/Users/colinsmith/miniconda3/envs/personal/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
INFO:policy_gradient:policy_gradient logger started.


In [ ]:
# create the tf session
sess = tf.InteractiveSession()

# Input
num_actions=3  # same as # of stocks
Variables=3
obs_dim=num_actions*Variables
NumOfLayers=3
architecture = "LSTM" # right now, valid inputs are LSTM and FFNN
LR="Adam"
actFunc="relu"
regulizer="l2"
regulizerScale=0.01


pg = policy_gradient.PolicyGradient(sess, obs_dim=obs_dim, 
                                    num_actions=num_actions,
                                    NumOfLayers=NumOfLayers, 
                                    LR=LR,
                                    architecture=architecture,
                                    actFunc=actFunc,
                                    learning_rate=1e-2,
                                    regulizer =regulizer,
                                    regulizerScale=regulizerScale
                                   )

# and now let's train it and evaluate its progress.  NB: this could take some time...
direc="/Users/colinsmith/mlp3/"
load_model=True
df,sf = pg.train_model(env, episodes=25001, log_freq=100, load_model=False,model_dir = direc)#, load_model=True)


Tensor("Mul:0", shape=(?, 3), dtype=float32)
(<tf.Tensor 'Mul:0' shape=(?, 3) dtype=float32>, <tf.Tensor 'fully_connected_1/BiasAdd:0' shape=(?, 3) dtype=float32>)
Tensor("tf_y:0", shape=(?, 3), dtype=float32)


INFO:policy_gradient:year #     0, mean reward: -42.3705, sim ret:   0.0000, mkt ret:   0.0000, net:   0.0000


[-42.37052938]


INFO:policy_gradient:year #   100, mean reward: -46.9257, sim ret:   0.0000, mkt ret:   0.0000, net:   0.0000


[-46.92568218]


### Results

Policy gradients beat the trading game!  That said, it doesn't work every time and it seems, looking at the charts below, as though it's a bit of a lucky thing.  But luck counts in the trading game as in life!


In [ ]:
sf['net'] = sf.simror - sf.mktror
#sf.net.plot()
sf.net.expanding().mean().plot()
sf.net.rolling(100).mean().plot()

In [ ]:
sf.net.rolling(100).mean().tail()